----
<img src="../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Access layer - Pricing streams
This notebook demonstrates how to use the pricing stream of the Access layer to retrieve streaming level 1 pricing data. The Access layer provides a simplified pricing streams interface designed for FinCoders. The complete pricing stream interface is available in the Content layer of the library.

#### About the open_pricing_stream function 
The **open_pricing_stream()** function allows you create and open a pricing stream object to retrieve streaming data via a single function call.


***open_pricing_stream()*** parameters: 
- ***universe***: Single instrument or list of instruments to subscribe to
    - Type: String or list
    - Optional: No  
- ***fields***: Single field or list of requested fields
    - Type: String or list
    - Optional: Yes  
- ***service***: Name of the streaming service publishing the instruments
    - Type: String
    - Optional: Yes  
- ***on_data***: Callback function to receive incoming data events
    - Type: Function
    - Optional: Yes  

**open_pricing_stream()** returns a PricingStream object.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

----

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../Configuration"

## Some Imports to start with

In [1]:
import refinitiv.data as rd
import datetime
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7ff6cf0dde20 {name='workspace'}>

## Retrieve data

### Using pricing streams with events

#### Define a callback to receive data events

In [3]:
def display_data(data, instrument, stream):
    clear_output(wait=True)
    current_time = datetime.datetime.now().time()
    print(current_time, "- Data received for", instrument)
    display(data)

#### Open the stream and register the callback

In [4]:
stream = rd.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID', 'ASK'],
    on_data=display_data
)

11:27:37.419180 - Data received for EUR=


,BID,ASK
EUR=,1.0983,1.0985


#### Close the stream

In [5]:
stream.close()

<OpenState.Closed: 3>

### Using pricing streams as a data cache

#### Create and open a Pricing stream object

The Pricing stream object is created for a list of instruments and fields. The fields parameter is optionnal. If you omit it, the Pricing stream will retrieve all fields available for the requested instruments

In [6]:
stream = rd.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID', 'ASK']
)

As soon as the method returns, the stream object is ready to be used. Its internal cache is constantly kept updated with the latest streaming information received from Eikon, Refinitiv Workspace or the Refinitiv Data Platform. All this happens behind the scene, waiting for your application to pull out data from the cache.  

#### Extract snapshot data from the streaming cache
Once the stream is opened, you can use the get_snapshot method to pull out data from its internal cache. get_snapshot can be called any number of times. As these calls return the latest received values, successive calls to get_snapshot may return different values. Returned DataFrames do not change in real-time, get_snapshot must be called every time your application needs fresh values. 

In [7]:
stream.get_snapshot()

,Instrument,BID,ASK
0,GBP=,1.3134,1.3138
1,EUR=,1.0981,1.0985
2,JPY=,122.77,122.8


#### Get a snapshot for a subset of instruments and fields

In [8]:
stream.get_snapshot(
    universe = ['EUR=', 'GBP='], 
    fields = ['BID', 'ASK']
)

,Instrument,BID,ASK
0,EUR=,1.0981,1.0983
1,GBP=,1.3133,1.3137


#### Other options to get values from the streaming cache

##### Direct access to real-time fields

In [9]:
print('GBP/BID:', stream['GBP=']['BID'])
print('EUR/BID:', stream['EUR=']['BID'])

GBP/BID: 1.3133
EUR/BID: 1.0981


##### Direct acces to a streaming instrument

In [10]:
gbp = stream['GBP=']
print(gbp['BID'])

1.3134


##### Iterate on fields

In [11]:
print('GBP=')
for field_name, field_value in stream['GBP=']:
    print('\t' + field_name + ': ', field_value)
    
print('JPY=')
for field_name, field_value in stream['JPY=']:
    print('\t' + field_name + ': ', field_value)

GBP=
	BID:  1.3133
	ASK:  1.3137
JPY=
	BID:  122.76
	ASK:  122.79


##### Iterate on streaming instruments and fields

In [12]:
for streaming_instrument in stream:
    print(streaming_instrument.name)
    for field_name, field_value in streaming_instrument:
        print('\t' + field_name + ': ', field_value)

GBP=
	BID:  1.3134
	ASK:  1.3135
EUR=
	BID:  1.0981
	ASK:  1.0983
JPY=
	BID:  122.76
	ASK:  122.79


#### Close the stream

In [13]:
stream.close()

<OpenState.Closed: 3>

### Close the session

In [14]:
rd.close_session()